In [197]:
import numpy as np
from math import *
import pandas as pd

# Plotting Graphs
import matplotlib.pyplot as plt

# Random Variable Simulation
from scipy import stats

# Remove Warnings
import warnings
warnings.filterwarnings('ignore')

In [198]:
'''TEST 1: ABSORPTION
======================'''
def fNAbsorption(F0, sigma1, eta, delta, timeSteps, expiry, numPaths):
    dt = expiry/timeSteps
    paths =  np.zeros((timeSteps+1, numPaths), np.float64)
    paths[0] = F0
    for t in range(1, timeSteps+1):
        rand1 = np.random.standard_normal(numPaths)
        rand1 = (rand1 - rand1.mean())/rand1.std()
        paths[t] = np.maximum((np.maximum(paths[t-1],0) 
                            + sigma1*np.power(np.maximum(paths[t-1],0)+delta, eta)*sqrt(dt)*rand1) ,0)
    return(paths)

In [199]:
'''TEST 2: REFLECTION
======================'''
def fNReflection(F0, sigma1, eta, delta, timeSteps, expiry, numPaths):
    dt = expiry/timeSteps
    paths =  np.zeros((timeSteps+1, numPaths), np.float64)
    paths[0] = F0 
    for t in range(1, timeSteps+1):
        rand1 = np.random.standard_normal(numPaths)
        rand1 = (rand1 - rand1.mean())/rand1.std()
        paths[t] = np.absolute(np.absolute(paths[t-1]) 
                            + sigma1*np.power(np.absolute(paths[t-1])+delta, eta)*sqrt(dt)*rand1)
    return(paths)

In [200]:
'''TEST 3: HIGHAM AND MAO
=========================='''
def fNHighamMao(F0, sigma1, eta, delta, timeSteps, expiry, numPaths):
    dt = expiry/timeSteps
    paths =  np.zeros((timeSteps+1, numPaths), np.float64)
    paths[0] = F0 
    for t in range(1, timeSteps+1):
        rand1 = np.random.standard_normal(numPaths)
        rand1 = (rand1 - rand1.mean())/rand1.std()
        paths[t] = np.absolute(paths[t-1] 
                            + sigma1*np.power(np.absolute(paths[t-1])+delta, eta)*sqrt(dt)*rand1)
    return(paths)

In [201]:
'''TEST 4: PARTIAL TRUNCATION
=============================='''
def fNPartialTrunc(F0, sigma1,eta, delta, timeSteps, expiry, numPaths):
    dt = expiry/timeSteps
    paths =  np.zeros((timeSteps+1, numPaths), np.float64)
    paths[0] = F0 
    for t in range(1, timeSteps+1):
        rand1 = np.random.standard_normal(numPaths)
        rand1 = (rand1 - rand1.mean())/rand1.std()
        paths[t] = np.maximum(paths[t-1] 
                            + sigma1*np.power(np.maximum(paths[t-1],0)+delta, eta)*sqrt(dt)*rand1,0)
    return(paths)

In [202]:
'''TEST 5: FULL TRUNCATION
==========================='''
def fNTrunc(F0, sigma1, eta, delta, timeSteps, expiry, numPaths):
    dt = expiry/timeSteps
    paths =  np.zeros((timeSteps+1, numPaths), np.float64)
    paths[0] = F0 
    for t in range(1, timeSteps+1):
        rand1 = np.random.standard_normal(numPaths)
        rand1 = (rand1 - rand1.mean())/rand1.std()
        paths[t] = np.maximum(paths[t-1] 
                            + sigma1*np.power(np.maximum(paths[t-1],0)+delta, eta)*sqrt(dt)*rand1, 0)
    return(paths)

In [203]:
'''TEST 6: LOG EULER
====================='''
def fNLogEuler(F0, sigma1, eta, delta, timeSteps, expiry, numPaths):
    dt = expiry/timeSteps
    paths =  np.zeros((timeSteps+1, numPaths), np.float64)
    paths[0] = F0 
    for t in range(1, timeSteps+1):
        rand1 = np.random.standard_normal(numPaths)
        rand1 = (rand1 - rand1.mean())/rand1.std()
        paths[t] = paths[t-1]*np.exp((np.power(paths[t-1]+delta, eta)/paths[t-1])*
                   (((-0.5 *(np.power(paths[t-1]+delta, eta)/paths[t-1])*pow(sigma1,2)*dt)+ 
                     sigma1*rand1*sqrt(dt))))
        
    return(paths)

In [67]:
loc = "Data\Calibration Data.xlsx"

# Obtain Swap Curve Data
swapData = pd.read_excel(loc,
                        sheet_name = "Swap Rates Data",
                        skiprows= [0, 1,3],
                       index_col = "Dates")

# Decimal Notation
swapData = pd.DataFrame.dropna(swapData, axis = 'rows')/100

'''SPOT CURVE CONSTRUCTION
==========================='''
# Curve on evaluation date: 17/02/2020
swapCurve = swapData.loc['2020-02-14']

swapMaturities = np.array(([i for i in range(1,31)] + # Years
                          [i*5 for i in range(7, 13)])) # 50 years

# Linear Interpolate swap curve
swapCurveInterp = np.interp(range(1, 61), swapMaturities, swapCurve)

# Bootstrap to obtain Zero Coupon Curve
zeroCoupon = [1]
for i in range(0, 60):
    zeroCoupon.append((1 - swapCurveInterp[i]*np.sum(zeroCoupon[:i]))/(1 + swapCurveInterp[i]))

forwardCurve = [zeroCoupon[i-1]/zeroCoupon[i]-1 for i in range(1, 61)]

In [255]:
'''CLOSED FORM CAP PRICER
=========================='''
def capletChiSquared(N, F0, K, expiry, sigma1, eta, delta):
    # Define parameters
    v = expiry*(pow(sigma1, 2))
    a = pow(K+delta, 2*(1 - eta))/(pow(1 - eta, 2)* v)
    b =1/(1 - eta)
    c = pow(F0+delta, 2*(1 - eta))/(pow(1 - eta, 2)* v)
    
    price = N*zeroCoupon[expiry]*((F0+delta)*(1- stats.ncx2.cdf(a, b+2, c)) - (K+delta)*stats.ncx2.cdf(c, b, a))
    return(price)

'''MONTE CARLO PRICER
========================='''
def monteCarloCap(N, F0, strike, expiry, sigma1, eta, delta, timeSteps, numPaths):
    steps = timeSteps * expiry
    fT = fNLogEuler(forwardCurve[int(expiry)],sigma1, eta, delta, steps, expiry, numPaths)[steps]
    price = N * zeroCoupon[expiry] * np.nanmean(np.maximum(fT - strike, 0))
    return(price)

In [ ]:
'Define the parameters'
N = 1e+5
delta = 0.01
sigma1 = 0.001
sigma2 = 0.001
eta = 0.45
strike = 0.001

# Simulation Parameters
expiry = 10
timeSteps = 252
numPaths = 10000

steps = timeSteps * expiry
paths = fNLogEuler(forwardCurve[int(expiry)],sigma1, eta, delta, steps, expiry, numPaths)[steps]
plt.plot(paths)
plt.show()

In [ ]:
np.nanmean(fNAbsorption(forwardCurve[int(expiry)],sigma1, eta, delta, timeSteps, expiry, numPaths)[timeSteps])
# forwardCurve[int(expiry)]

In [ ]:
# Compute Cap Values
print(capletChiSquared(N, forwardCurve[int(expiry)], strike, expiry, sigma1,eta, delta))
print(monteCarloCap(N,forwardCurve[int(expiry)],strike, expiry, sigma1, eta, delta, timeSteps, numPaths ))

In [ ]:
# Simulate under discretization schemes
# absorption = fNAbsorption(forwardCurve[int(expiry)],sigma1, eta, delta, timeSteps, expiry, numPaths)
# reflection = fNReflection(forwardCurve[int(expiry)],sigma1, eta, delta, timeSteps, expiry, numPaths)
# highamMao = fNHighamMao(forwardCurve[int(expiry)],sigma1, eta, delta, timeSteps, expiry, numPaths)
# partialTrunc = fNPartialTrunc(forwardCurve[int(expiry)],sigma1, eta, delta, timeSteps, expiry, numPaths)
# fullTrunc = fNTrunc(forwardCurve[int(expiry)],sigma1, eta, delta, timeSteps, expiry, numPaths)
# logEuler = fNLogEuler(forwardCurve[int(expiry)],sigma1, eta, delta, timeSteps, expiry, numPaths)